In [2]:
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import json
import pickle
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from keras.optimizers import SGD
import random
from keras.models import load_model

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [6]:
print("~This is words list~")
print(words[3:5])
print("-"*50)
print("~This is documents list~")
print(documents[3:5])
print("-"*50)
print("~This is classes list~")
print(classes[3:10])

~This is words list~
['ada', 'orang']
--------------------------------------------------
~This is documents list~
[(['Hi', 'there'], 'greetings'), (['Hello'], 'greetings')]
--------------------------------------------------
~This is classes list~
['evening', 'night', 'goodbye', 'thanks', 'no-response', 'neutral-response', 'about']


In [7]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print("~Document Length~")
print(len(documents), "documents\n\n")
print("-"*100)

print("~Class Length~")
print(len(classes), "classes\n\n", classes)
print("-"*100)

print("~Word Length~")
print(len(words), "unique lemmatized words\n\n", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

~Document Length~
310 documents


----------------------------------------------------------------------------------------------------
~Class Length~
113 classes

 ['about', 'afternoon', 'anxious', 'ask', 'casual', 'creation', 'death', 'default', 'depressed', 'dialog.holdon', 'dialog.hug', 'dialog.idontcare', 'dialog.introduction', 'dialog.sorry', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greetings', 'greetings.bye', 'greetings.day', 'greetings.hello', 'greetings.howareyou', 'greetings.islam', 'greetings.nicetomeetyou', 'greetings.nicetoseeyou', 'greetings.nicetotalktoyou', 'greetings.night', 'greetings.noon', 'greetings.whatareyoudoing', 'happy', 'hate-me', 

In [8]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


print("Training data created")

Training data created


In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

adam_optimizer = Adam()
# sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist)
print("\n")
print("*" * 50)
print("\nModel Created Successfully")


Epoch 1/200
62/62 [==============================] - 1s 4ms/step - loss: 4.7208 - accuracy: 0.0323
Epoch 2/200
62/62 [==============================] - 0s 5ms/step - loss: 4.5747 - accuracy: 0.0677
Epoch 3/200
62/62 [==============================] - 0s 4ms/step - loss: 4.3248 - accuracy: 0.0968
Epoch 4/200
62/62 [==============================] - 0s 5ms/step - loss: 4.0846 - accuracy: 0.1258
Epoch 5/200
62/62 [==============================] - 0s 4ms/step - loss: 3.7701 - accuracy: 0.1903
Epoch 6/200
62/62 [==============================] - 0s 5ms/step - loss: 3.4168 - accuracy: 0.2839
Epoch 7/200
62/62 [==============================] - 0s 5ms/step - loss: 2.9925 - accuracy: 0.3194
Epoch 8/200
62/62 [==============================] - 0s 5ms/step - loss: 2.7676 - accuracy: 0.3839
Epoch 9/200
62/62 [==============================] - 0s 5ms/step - loss: 2.4904 - accuracy: 0.4581
Epoch 10/200
62/62 [==============================] - 0s 5ms/step - loss: 2.0988 - accuracy: 0.5387
Epoch 11/

In [12]:
model = load_model('chatbot.h5')
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [13]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [14]:
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print("found in bag: %s" %w)
  return(np.array(bag))

In [15]:
def predict_class(sentence, model):
  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  error = 0.25
  results = [[i, r] for i, r in enumerate(res) if r>error]

  results.sort(key=lambda x:[1], reverse=True)
  return_list= []

  for r in results:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list

In [16]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if(i['tag']== tag):
      result = random.choice(i['responses'])
      break
  return result

def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

In [21]:
def start_chat():
  print("Bot: Rhea is here! How can help you?.\n\n")
  while True:
    inp = str(input()).lower()
    if inp.lower()=="end":
      break
    if inp.lower()== '' or inp.lower()== '*':
      print('Please re-phrase your query!')
      print("-"*50)
    else:
      print(f"Bot: {chatbot_response(inp)}"+'\n')
      print("-"*50)

In [23]:
start_chat()

Bot: Rhea is here! How can help you?.


halo
1/1 [==============================] - 0s 105ms/step
Bot: Halo. Katakan padaku bagaimana perasaanmu hari ini?

--------------------------------------------------
end


In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the Keras model
model = load_model('chatbot.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('chatbot.tflite', 'wb') as f:
    f.write(tflite_model)

print("\n")
print("*" * 50)
print("\nModel Converted to TensorFlow Lite Successfully")




**************************************************

Model Converted to TensorFlow Lite Successfully
